# Evaluation

In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import evaluate_segments
import random
from src.Video import Video
from src.Segment import Segment
from src import search
import cv2

## Parameters

In [76]:
NUM_VIDEOS = 2
GRID_SIZE = 2
BINS = [180, 180]
HIST_FRAME_SKIP = 20
REFRESH = False

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set

In [77]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 2 - Grid 2 - Bins [180, 180] - Skip 20
Loading / processing dataset...
Done processing!

In [78]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:      {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms:      {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

TRAINING SET:
Num. videos:   2
Num. segments: 189
Duration:      529.7 s
Num frames:      14542
Num histograms:      827


## Select random test set

In [79]:
test_n_segments = 100
test_set = []
labels = []

for i in range(test_n_segments):
    
    # Find random video
    video = random.choice(training_set)
    
    # Select random segment and add histogram to test set
    segment = random.choice(video.segments)
    test_set.append(segment.histograms)
    labels.append(segment)

In [80]:
# Print statistics
print("TEST SET:")
print("Num. histograms: {:d}".format( np.sum([len(histogram) for histogram in test_set]) ))

TEST SET:
Num. histograms: 375


<br>

## Run model on test set

In [81]:
printParams()

for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    print('{}'.format(method))
    %timeit -n 10 search.findFrame(test_set[0][0], training_set, method)

for ch in [[0], [1], [0, 1]]:
    print('{}'.format(ch))
    %timeit -n 10 search.findFrame(test_set[0][0], training_set, cv2.HISTCMP_CORREL, channels=ch)


# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CORREL)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_INTERSECT)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_BHATTACHARYYA)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_KL_DIV)

Num. Vid 2 - Grid 2 - Bins [180, 180] - Skip 20
0
3.77 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1
4.7 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2
4.8 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3
4.34 ms ± 563 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
4
6.38 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
5
5.1 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[0]
2.01 ms ± 67.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[1]
2.06 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[0, 1]
3.55 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [82]:
printParams()

results = []

for i, segment_histograms in enumerate(test_set):
    print('\rSearching segment {}/{}'.format(i+1, len(test_set)), end='', flush=True)
    
    x = random.choice(range(len(segment_histograms)))
    results.append(search.findFrame(segment_histograms[0], training_set, 5, cv2.HISTCMP_CHISQR_ALT))

Num. Vid 2 - Grid 2 - Bins [180, 180] - Skip 20
Searching segment 100/100

## Evaluate performance

In [83]:
evaluate_segments(results, labels)

Segment evaluation:
Correct: 0
Wrong:   100
Total:   100
TPR:     0.0%


# Manual Evaluation

In [73]:
#Manually check what happens
test_vid = 1
for i in range(len(training_set[test_vid].segments)):
    
    hists = training_set[test_vid].segments[i].histograms

    #Possibly shrink set for readability
    # train = training_set.copy()
    # for t in train:
    #     t.segments = t.segments[:20]

    # 0 = first histogram of segment, so perfect match
    tf = 0
    search.findFrame(hists[tf], train, cv2.HISTCMP_CHISQR_ALT, 5, prints=False, printRes = True)


video 00001 - segment 109
video 00002 - segment 1
video 00002 - segment 2
video 00002 - segment 3
video 00002 - segment 4
video 00002 - segment 5
video 00002 - segment 6
video 00002 - segment 7
video 00002 - segment 8
video 00002 - segment 9
video 00002 - segment 10
video 00002 - segment 11
video 00002 - segment 12
video 00002 - segment 13
video 00002 - segment 14
video 00002 - segment 15
video 00002 - segment 16
video 00002 - segment 17
video 00002 - segment 18
video 00002 - segment 19
video 00002 - segment 20
video 00002 - segment 21
video 00002 - segment 22
video 00002 - segment 23
video 00002 - segment 24
video 00002 - segment 25
video 00002 - segment 26
video 00002 - segment 27
video 00002 - segment 28
video 00002 - segment 29
video 00002 - segment 30
video 00002 - segment 31
video 00002 - segment 32
video 00002 - segment 33
video 00002 - segment 34
video 00002 - segment 35
video 00002 - segment 36
video 00002 - segment 37
video 00002 - segment 38
video 00002 - segment 39
video 00